# Plotting CESM output

---

Maps of CESM output (chemical compounds)

Maps of CESM-SE output, and regional refinement projections

Zonal average contour plots (including calculating ZA)

Difference maps - how to choose different color bars

Panel map plots with different legends

Box-plots (percentiles)

Polar graph (e.g. wind rose)

Multi-panel graph (customizing sizes and locations of each panel)

Add flow stream lines on contour map

---

# Plotting other things
Plots of model results with aircraft observations on top (maps, curtains)